# Домашнее задание 1: LLM Usage

**ФИО:** 

**Дедлайн:** 3 декабря в 23:59

Для выполнения задания мы дадим вам ключи для API провайдеров (OpenAI, DeepSeek, Together AI). Они привязаны к аккаунтам с фиксированным балансом. Если вы будете слать очень много запросов, в том числе и в цикле, или очень большие по длине запросы, деньги закончатся у всей группы, и никто не сможет доделать домашку. Мы, конечно, что-нибудь придумаем, но лучше этого не допускать :) Поэтому не размечайте/генерируйте большое количество примеров и не запускайте запросы по API в цикле (это не относится к последнему заданию, где вам надо работать с локальной моделькой в Колабе).

**Не публикуйте ключи**. Никогда не вставляйте ключ прямым текстом в ячейку с кодом, если планируете кому-то показывать тетрадку или выкладывать её на публичный GitHub (и если не планируйте, лучше тоже не вставляйте — хорошие практики кода лучше усваивать сразу).

* В Google Colab используйте вкладку "Secrets" (ключик слева).
* При локальном запуске используйте файл `.env` и библиотеку `python-dotenv`. Пример файла `.env` есть в репо, можете просто переименовать `.env.example` в `.env` и вставить ключи.

В ячейках ниже настроена загрузка ключей в зависимости от того, где вы запускаете код.

In [ ]:
!pip3 install --upgrade tiktoken openai python-dotenv transformers accelerate bitsandbytes -q

In [ ]:
# Для Google Colab

from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

In [ ]:
# Если запускаете тетрадку у себя

import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')
TOGETHER_API_KEY = os.getenv('TOGETHER_API_KEY')

In [ ]:
# API-эндпойнты для DeepSeek и Together AI
DEEPSEEK_BASE_URL = "https://api.deepseek.com"
TOGETHER_BASE_URL = "https://api.together.xyz/v1"

## Задание 1. Токенизация (2 балла)

Как мы обсуждали на семинаре, LLM видят текст не по буквам или словам, а по токенам. Для английского языка 1 токен — это часто целое слово, а для других языков (особенно с отличной от латиницы письменностью) одно слово может распадаться на несколько токенов. Поскольку API обычно тарифицируются за токены, работа с некоторыми языками может быть дороже и медленнее, да и перформанс моделек зависит от эффективности токенизации.

1. Найдите три текста на разных языках.
    * Тексты должны быть примерно одинаковыми по объёму (например, одна и та же новость или статья из Википедии, или просто три куска текста длиной не менее нескольких абзацев каждый).
    * Языки должны быть непохожи друг на друга и на английский. Можно брать языки с отличающимися письменностями (например: кириллица, латиница, иероглифы, арабская письменность, грузинский и т.д.), а можно и с одной, но из разных семей. *Обоснуйте свой выбор языков в комментарии*.
2. Выберите один токенизатор: либо `tiktoken` (токенизатор для моделей OpenAI), либо токенизатор любой открытой модели (например, Qwen или Llama — их можно загрузить через `transformers`.
3. Для каждого текста посчитайте количество символов. Затем токенизируйте и посчитайте количество токенов. Вычислите отношение `Tokens / Characters`.
4. Сделайте вывод. Какой язык токенизируется эффективнее всего, а какой самый неэффективный? Предположите, почему результат именно такой.

## Задание 2. Синтетический датасет (2 балла)

LLM'ки часто используют для создания датасетов. Представьте ситуацию: вам нужно сделать модельку кластеризации или классификации для задачи разрешения лексической многозначности (Word Sense Disambiguation). Размечать тысячи примеров вручную — долго и дорого, поэтому хорошей идеей будет попросить модель сгенерировать синтетический датасет, который потом можно валидировать.

Сгенерируйте набор из 5 пар предложений с омонимами на русском языке. В каждой паре должно быть одно и то же слово, употреблённое в разных (двух) значениях (например, «ключ от двери» и «ключ — родник»). Для простоты примем, что мы рассматриваем только два любых значения для каждого слова, и для каждого нужно по одному предложению.

LLM'ка должна отдать результат в формате JSON. Структура может быть любой удобной для вас, но она должна быть валидной и машиночитаемой. Примерная схема может выглядеть так (но вы можете придумать любую другую):

```json
{"homonyms": [
  {
    "word": "коса",
    "meanings": [
      {
        "definition": "Сельскохозяйственный инструмент",
        "sentence": "Острая коса легко срезала высокую траву на лугу."
      },
      {
        "definition": "Вид укладки волос",
        "sentence": "Девушка заплела длинную русую косу."
      }
    ]
  },
  ...
]}
```

Проведите два эксперимента:

1.  С **закрытой моделью** (например, `gpt-5.1` или `gpt-5-mini`).
2.  С **открытой моделью**: например, `deepseek-chat` или любой другой семейства Qwen или Gemma через Together AI API. Можно использовать что-то из свежего: `Qwen/Qwen3-Next-80B-A3B-Instruct`, `Qwen/Qwen3-235B-A22B-Instruct-2507-tput` или маленькие модели, вроде `Qwen/Qwen2.5-14B-Instruct` (или даже 7B) или `google/gemma-3-4b-it`.

Можете дать модели список из пяти омонимичных слов, тогда ей нужно будет только определить значения и придумать примеры, а можете не подсказывать и посмотреть, какие слова предложит сама модель — так интереснее!

Для получения чистого JSON вы можете просто попросить модель не писать лишнего текста, кроме JSON (возможно, придётся убирать знаки начала и конца блока кода), либо, если интересно понять, как делать профессионально, разобраться со [structured output](https://platform.openai.com/docs/guides/structured-outputs) или более простым, но устаревшим и не всеми поддерживаемым [JSON output](https://api-docs.deepseek.com/guides/json_mode)

В конце сравните результаты и сделайте выводы. Какая модель лучше справилась с подбором омонимов и/или примеров? Были ли трудности заставить модель отвечать в нужном формате? Были ли галлюцинации в значениях слов?


## Задание 3. Перевод (4 балла)

В этом задании мы посмотрим, как параметры генерации и промпт-инжиниринг влияют на качество перевода.

Выберите текст на каком-нибудь иностранном языке, отличном от английского (только не очень большой). Это может быть французский, немецкий, финский или любой другой, но желательно не очень популярный, чтобы задачка была поинтереснее (например, малый язык, если вы знаете какой-нибудь).

Вам нужно перевести этот текст на русский язык, используя две модели: одну проприетарную и одну открытую.

Для каждой из двух моделей проведите три эксперимента:

1.  **Набор гиперпараметров 1.** Выберите параметры `temperature` и `top_p` (какой-то один или комбинацию, на ваше усмотрение, например, высокую температуру для большей креативности).
2.  **Набор гиперпараметров 2.** Измените параметры (например, сделайте генерацию более детерминированной).
3.  **Лучшие настройки + Промпт-инжиниринг.** Возьмите тот набор параметров из двух предыдущих, который показал лучший результат, и попробуйте улучшить качество с помощью изменения промпта. Выберите один из способов:
      * *Few-shot:* добавьте в контекст несколько примеров удачного перевода с этого языка.
      * *Error Correction:* добавьте в промпт указания на ошибки, которые модель допустила в предыдущих попытках (например, «не переводи названия буквально» или «сохраняй официальный стиль»). Может быть хорошей идеей добавить правильный перевод имён или специфических терминов, если они есть у вас в тексте.

Итого у вас должно получиться 6 вариантов перевода (2 модели × 3 эксперимента).

Выведите результаты всех экспериментов. Ответьте на вопросы:

  * Какая модель справилась лучше?
  * Как изменение гиперпараметров повлияло на текст? Стал ли он точнее, литературнее или, наоборот, начал разваливаться?
  * Помогло ли усложнение промпта в третьем эксперименте улучшить итоговый результат?

## Задание 4\*. Классификация (2 балла)

В этом задании вам нужно будет запустить LLM'ку локально (в Google Colab) и проверить, как она справляется с задачей определения грамматичности предложений на русском языке.

Мы будем использовать датасет [RuCoLA](https://huggingface.co/datasets/RussianNLP/rucola) (Russian Corpus of Linguistic Acceptability). Это набор предложений, часть из которых грамматически верные (Acceptable, класс 1), а часть — содержат ошибки (Unacceptable, класс 0). Ошибки могут быть разными: от нарушения согласования до галлюцинаций моделей.

В оригинальном бенчмарке RuCoLA тестовый сет не содержит лейблов, а валидационный разбит на `in_domain` (примеры из лингвистических публикаций) и `out_of_domain` (генерация нейросетей). Мы будем использовать **in\_domain**, потому что он понятнее и чище.


In [1]:
import pandas as pd

train_url = "https://huggingface.co/datasets/RussianNLP/rucola/resolve/main/data/in_domain_train.csv"
dev_url = "https://huggingface.co/datasets/RussianNLP/rucola/resolve/main/data/in_domain_dev.csv"

df_train = pd.read_csv(train_url)
df_dev = pd.read_csv(dev_url)

print(f"Train shape: {df_train.shape}")
print(f"Dev shape: {df_dev.shape}")

df_dev.head()

Train shape: (7869, 5)
Dev shape: (983, 5)


,id,sentence,acceptable,error_type,detailed_source
0,0,Иван вчера не позвонил.,1,0,Paducheva2013
1,1,"У многих туристов, кто посещают Кемер весной, ...",0,Syntax,USE8
2,2,Лесные запахи набегали волнами; в них смешалос...,1,0,USE5
3,3,Вчера президент имел неофициальную беседу с ан...,1,0,Seliverstova
4,4,Коллега так и не признал вину за катастрофу пе...,1,0,Testelets


**Что нужно сделать:**

1.  **Подготовка данных.** Из валидационной выборки возьмите случайные 50–100 примеров (это будет наш тест). Не берите сильно больше, иначе вы будете ждать результатов очень долго — инференс в Колабе небыстрый.
      * *Важно:* Убедитесь, что в вашей выборке присутствуют оба класса (и 0, и 1), желательно поровну.
2.  **Выбор модели.** Вам нужно выбрать и загрузить одну Instruct-модель, которая влезет в память Colab. Можно использовать одну из этих или поискать что-то другое до 7B включительно:
      * [Qwen/Qwen3-4B-Instruct-2507](https://huggingface.co/Qwen/Qwen3-4B-Instruct-2507)
      * [Qwen/Qwen2.5-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-7B-Instruct)
      * [unsloth/Llama-3.2-3B-Instruct](https://huggingface.co/unsloth/Llama-3.2-3B-Instruct)
      * [t-tech/T-lite-it-1.0](https://huggingface.co/t-tech/T-lite-it-1.0) — модель, специально дообученная для русского языка
      * *Не берите* модели с "Thinking" для этой задачи — они генерируют длинные цепочки мыслей перед ответом, будет работать очень долго.
3.  **Эксперимент 1: Zero-shot.**
      * Напишите промпт, в котором вы подаёте модели предложение и просите классифицировать его
      * Прогоните ваши тестовые примеры через модель
      * Распарсите ответ, достаньте оттуда метку класса в формате, который вы задали. Если модель не слушается инструкций и отвечает не так, как вы её попросили, это можно считать за ошибку
4.  **Эксперимент 2: Few-shot.**
      * Возьмите 5–10 примеров из трейна. Следите, чтобы там были и грамматичные, и неграмматичные предложения
      * Включите эти примеры с правильными ответами в промпт
      * Прогоните те же тестовые примеры
5.  **Итоги.**
      * Посчитайте метрики классификации (Accuracy или F1) для обоих экспериментов
      * Выведите несколько примеров ошибочно классифицированных предложений, проанализируйте вручную. Если хотите, можно использовать столбец `error_type` в оригинальном датасете и собрать статистику, какие типы ошибок распознаются неправильно чаще, но это опционально
      * Сделайте выводы: помог ли few-shot? Насколько вообще маленькая модель справляется с этой задачей Есть ли какие-то очевидные закономерности в ошибках модели?

> **Важно:** не пытайтесь делать это задание через модели по API, оно не будет засчитано и вы можете быстро израсходовать бюджет ключей.

**Подсказки:**

1. Не надо подавать в модель все тестовые примеры сразу, нужно по одному на каждый запрос
2. Можно поставить очень маленькое значение параметра `max_new_tokens` — если модель не будет слушаться инструкций и начнёт писать пояснения или рассуждения, она по крайней мере быстро остановится и зря не потратит время и память
3. В этом задании не обязательно использовать вывод в JSON, хотя можно

## Расскажите нам, что думаете об этой домашке

Как вам задания? Насколько интересно? Насколько сложно? Что хотелось бы добавить, изменить или убрать? Только честно! :)